In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from tqdm import trange

from transformer_model import Transformer, SelfAttention, MLP

In [2]:
dims = 5
seq_len = 20

In [4]:
def generate_data(batch_size, dims, seq_len):
    w = np.random.normal(0, 1, size=(batch_size, dims))
    x = np.random.normal(0, 1, size=(batch_size, seq_len, dims))
    y = np.einsum("bp, blp -> bl", w, x)
    assert y.shape == (batch_size, seq_len)
    interweaved = []
    for b in range(batch_size):
        batch = []
        for s in range(seq_len):
            batch.append(x[b, s, :])
            batch.append(y[b, s, :])
        interweaved.append(batch)
    interweaved = np.array(interweaved)
    interweaved = interweaved[:, :-1, :]
    interweaved = torch.from_numpy(interweaved).to(torch.float)
    return x, y, w, interweaved

In [5]:
transformer = Transformer(
    num_layers=4,
    input_dim=dims,
    num_heads=1,
    mlp_hidden_dim=16
)

optimizer = torch.optim.SGD(transformer.parameters(), lr=0.01)
for _ in trange(200):
    x, y, w, interweaved, label = generate_data(8, dims, seq_len)
    output = transformer(interweaved)
    pred = output[:, -1, :]
    loss = torch.mean((pred - label) ** 2)
    assert not torch.isnan(loss), loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  0%|          | 0/200 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 5, got 4)

In [6]:
transformer = Transformer(
    num_layers=4,
    input_dim=dims,
    num_heads=1,
    mlp_hidden_dim=16
)
x, y, w, interweaved, label = generate_data(8, dims, seq_len)
output = transformer(interweaved)

print(output)

ValueError: not enough values to unpack (expected 5, got 4)

In [225]:
transformer.eval()
with torch.no_grad():
    x, y, w, interweaved, label = generate_data(dims, seq_len)
    pred = transformer(interweaved)[-1, :]
    loss = torch.mean((pred - label) ** 2)
    print(pred, label, loss)

tensor([0.0033, 0.0093]) tensor([-0.3360,  0.1111]) tensor(0.0627)
